In [ ]:
# Derin Öğrenmenin Temelleri Proje.
# Defect Detection in Industrial Products.
# PCB

# Tespit Edebileceklerimiz
"""
0 – mouse_bite:
PCB kenarında fare ısırığı gibi küçük çentikler.

1 – spur:
İstenmeyen ince bakır çıkıntısı (kıl gibi uzantı).

2 – missing_hole:
Açılması gereken delik/via’nın hiç açılmamış olması.

3 – short:
İki bakır hattın kazara birleşip kısa devre oluşturması.

4 – open_circuit:
Bir bakır hattın kopması; bağlantının kesilmesi.

5 – spurious_copper:
Kart üzerinde gereksiz, şemaya ait olmayan bakır parçaları.
"""

'\n0 – mouse_bite:\nPCB kenarında fare ısırığı gibi küçük çentikler.\n\n1 – spur:\nİstenmeyen ince bakır çıkıntısı (kıl gibi uzantı).\n\n2 – missing_hole:\nAçılması gereken delik/via’nın hiç açılmamış olması.\n\n3 – short:\nİki bakır hattın kazara birleşip kısa devre oluşturması.\n\n4 – open_circuit:\nBir bakır hattın kopması; bağlantının kesilmesi.\n\n5 – spurious_copper:\nKart üzerinde gereksiz, şemaya ait olmayan bakır parçaları.\n'

In [ ]:
# LINUX ORTAMINDAYIZ

!uname -a


Linux 7ce3b6aa79b9 6.6.105+ #1 SMP Thu Oct  2 10:42:05 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
!nvidia-smi

Wed Dec 24 07:29:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # drive'mizi colab'a bağlıyoruz.

Mounted at /content/drive


In [ ]:
# Drive'dan /content'e komple kopyala
!cp -r "/content/drive/MyDrive/Burcu Hoca Proje/Project/pcb-defect-dataset" "/content/pcb-defect-dataset"


In [ ]:
import yaml

yaml_yolu = "/content/pcb-defect-dataset/data.yaml"

with open(yaml_yolu, "r") as f:
    data = yaml.safe_load(f)

# Drive yerine content dizinini gösterelim
data["path"] = "/content/pcb-defect-dataset"
data["train"] = "train"
data["val"] = "val"
data["test"] = "test"

with open(yaml_yolu, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("✅ data.yaml güncellendi!")
print(open(yaml_yolu).read())


✅ data.yaml güncellendi!
path: /content/pcb-defect-dataset
train: train
val: val
test: test
names:
  0: mouse_bite
  1: spur
  2: missing_hole
  3: short
  4: open_circuit
  5: spurious_copper



In [ ]:
%cd /content
!git clone https://github.com/iMoonLab/yolov13.git
%cd yolov13

# requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""torch>=2.2.0
torchvision>=0.17.0
torchaudio>=2.2.0
tqdm
PyYAML
numpy
opencv-python>=4.8.0
pillow>=10.0.0
matplotlib
scipy
ultralytics>=8.3.63
supervision>=0.22.0
onnx==1.14.0
onnxruntime-gpu>=1.17.1
albumentations>=1.3.1
psutil
gradio""")

!pip install -r requirements.txt
!pip install -e .


/content
Cloning into 'yolov13'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 527 (delta 135), reused 116 (delta 116), pack-reused 229 (from 2)
Receiving objects: 100% (527/527), 29.91 MiB | 16.91 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/yolov13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 121.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8

In [ ]:
import shutil, os

base = "/content/yolov13/ultralytics/cfg/models/v13/yolov13.yaml"
target_dir = os.path.dirname(base)

for variant in ["n", "s", "l", "x"]:
    new_yaml = os.path.join(target_dir, f"yolov13{variant}.yaml")
    shutil.copy(base, new_yaml)
    with open(new_yaml, "r") as f:
        lines = f.readlines()
    with open(new_yaml, "w") as f:
        f.write(f"# Auto-generated {variant} model\nscale: \"{variant}\"\n")
        f.writelines(lines)
    print("✅ Created:", new_yaml)


✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13n.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13l.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13x.yaml


In [ ]:
# Drive bağlantısı
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

from ultralytics import YOLO

# Model
model = YOLO("/content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml")

# --- A100 80GB ayarlaması-
results = model.train(
    data="/content/pcb-defect-dataset/data.yaml",
    epochs=50,
    imgsz=640,

    # otomatik batch size
    batch=-1,

    device="cuda",

    # 2. Hızlandırma Ayarları
    workers=16,      # CPU GPU'ya veri yetiştirmek için kullanacak,çekirdek sayısı
    cache=True,      # RAM'den oku

    # 3. Drive kaydetme
    project="/content/drive/MyDrive/YOLO_Egitimleri",
    name="pcb_yolo13s_A100_Final",
    save=True,
    save_period=1,   # Her epoch'ta

    # Optimizasyon Ayarların
    optimizer="SGD", # optimizasyon algoritmamız
    lr0=0.01, # öğrenme hızı
    momentum=0.937, # modelin önceki adımlarla bağlı olmasını sağlar
    cos_lr=True, # learning rateyi dalgalı şekilde yürütür
    close_mosaic=10, # Son 10 epochta 4 farklı resim üzerinden eğitim metodunu kapatır
    patience=20, # erken durdurma,ancak geniş optimizasyon aralığında
    amp=True # hesaplamaları hassas yapması için
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/yolov13/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml, data=/content/pcb-defect-dataset/data.yaml, epochs=50, time=None, patience=20, batch=-1, imgsz=640, save=True, save_period=1, cache=True, device=cuda, workers=16, project=/content/drive/MyDrive/YOLO_Egitimleri, name=pcb_yolo13s_A100_Final, exist_ok=False, pretrained=True, optimizer=SGD, verbose=Tr

100%|██████████| 755k/755k [00:00<00:00, 184MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1      9344  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2, 1, 2]          
  2                  -1  1     20800  ultralytics.nn.modules.block.DSC3k2          [64, 128, 1, False, 0.25]     
  3                  -1  1     37120  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2, 1, 4]        
  4                  -1  1     78464  ultralytics.nn.modules.block.DSC3k2          [128, 256, 1, False, 0.25]    
  5                  -1  1     68352  ultralytics.nn.modules.conv.DSConv           [256, 256, 3, 2]              
  6                  -1  2    677120  ultralytics.nn.modules.block.A2C2f           [256, 256, 2, True, 4]        
  7                  -1  1    134400  ultralytics

100%|██████████| 10.1M/10.1M [00:00<00:00, 317MB/s]


AMP: checks passed ✅


...

      50/50      54.4G       1.76      1.585      1.285         61        640: 100%|██████████| 79/79 [01:09<00:00,  1.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.00s/it]

                   all       1066       2164      0.971      0.462      0.501      0.308      0.297



50 epochs completed in 1.070 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Final/weights/last.pt, 18.6MB
Optimizer stripped from /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Final/weights/best.pt, 18.6MB

Validating /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Final/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLOv13s summary: 535 layers, 9,003,241 parameters, 0 gradients, 20.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


                   all       1066       2164      0.969       0.46      0.501       0.31      0.297
            mouse_bite        183        372      0.982      0.473       0.51      0.317      0.305
                  spur        179        369       0.94      0.388      0.432      0.267      0.257
          missing_hole        165        331       0.99       0.45      0.459      0.332      0.292
                 short        201        411      0.968      0.521      0.578       0.36      0.333
          open_circuit        178        359      0.963      0.507      0.552      0.264      0.301
       spurious_copper        160        322      0.967      0.422      0.471      0.319      0.294
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Final


In [ ]:
import torch, gc
# gpu memory boşalt
def clear_gpu_memory():
    print("GPU memory'sini boşalt")
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("✅ Kalan kullanım:",
          f"{torch.cuda.memory_allocated()/1024**2:.1f} MB / {torch.cuda.get_device_properties(0).total_memory/1024**2:.1f} MB")

clear_gpu_memory()


In [ ]:
# RAM boşaltmak için,gerekli durumlarda.

import os
os.kill(os.getpid(), 9)
